In [3]:
#CrossCompute

Zipcode = 11419


In [2]:
import pandas as pd
from sodapy import Socrata # Used to access/ work with NYCOpenData API
import folium

# Grabbing data from API
client = Socrata("data.cityofnewyork.us",
                'YFHnlAd1f74IprxACGOlr46td',
                username="nycopendataninjas@gmail.com",
                password="DataNinjas4TheWin!")

# Limits the data to only trees that are ALIVE in that specified zipcode that was entered above^
results = client.get("fhrw-4uyv", incident_zip = Zipcode,complaint_type="Sewer",
                     descriptor = "Catch Basin Clogged/Flooding (Use Comments) (SC)",
                     limit=10000)

# Convert to pandas DataFrame
df_threeOneOneReq = pd.DataFrame.from_records(results)

# Only get the columns that are useful
df_threeOneOneReq[['latitude','longitude']]

,latitude,longitude
0,40.68465116557191,-73.83008135606646
1,40.68405687258566,-73.83285917965487
2,40.690636196427924,-73.8291862515534
3,40.69161321630308,-73.83666620882741
4,40.685837578189094,-73.8268476319625
5,40.685611053802326,-73.83329878273523
6,40.69563833902324,-73.81570779755315
7,40.69359308736717,-73.82970516862927
8,40.68750828614852,-73.81229395493435
9,40.69066509760756,-73.83203848700488


In [29]:
client = Socrata("data.cityofnewyork.us",
                'YFHnlAd1f74IprxACGOlr46td',
                username="nycopendataninjas@gmail.com",
                password="DataNinjas4TheWin!")

# Limits the data to only trees that are ALIVE in that specified zipcode that was entered above^
results = client.get("5rq2-4hqu", zipcode = Zipcode, status = 'Alive', limit=10000)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

# Only get the columns that are useful
results_df = results_df[['tree_dbh', 'health','status','latitude','longitude','spc_latin']]            
            
# Replaces words with numbers so that it is easier to create a 'grade' for each tree
results_df = results_df.replace(['Poor','Fair','Good'],[0,50,100])

# 'tree_dbh' was an object, this converts it to an int so that it can be added to 'health' and 'status'
results_df['tree_dbh'] = pd.to_numeric(results_df['tree_dbh'])

# Anywhere there is an 'NaN', make it a zero
results_df = results_df.fillna(0)

df = pd.read_csv('Species_Types.csv')
df = df.set_index('Species')

results_df['Type'] = df.loc[results_df.spc_latin,'Type'].values

results_df = results_df.replace(['deciduous','coniferous','evergreen','both'],[1,0,0,0])

# Generates a final grade that will be the value of the weight on the heat map for each tree
results_df['Final Grade'] = ((results_df.tree_dbh + results_df.health)/100)*results_df.Type

# Removes all the trees that dont lose leaves
results_df = results_df[results_df.Type != 0]
results_df = results_df.fillna(0)

results_df

,tree_dbh,health,status,latitude,longitude,spc_latin,Type,Final Grade
0,13,100,Alive,40.69199129,-73.82114902,Metasequoia glyptostroboides,1,1.13
1,4,50,Alive,40.69161541,-73.82096547,Metasequoia glyptostroboides,1,0.54
2,3,100,Alive,40.69153616,-73.82092678,Ulmus americana,1,1.03
3,15,100,Alive,40.69210017,-73.82832551,Gleditsia triacanthos var. inermis,1,1.15
4,18,50,Alive,40.69166655,-73.829862,Gleditsia triacanthos var. inermis,1,0.68
5,12,100,Alive,40.6921299,-73.82821961,Gleditsia triacanthos var. inermis,1,1.12
6,24,100,Alive,40.69382431,-73.82753579,Quercus palustris,1,1.24
7,21,50,Alive,40.69385554,-73.82742485,Quercus palustris,1,0.71
8,4,100,Alive,40.69394083,-73.8271218,Quercus palustris,1,1.04
9,26,100,Alive,40.69379328,-73.82764605,Quercus palustris,1,1.26


In [6]:
treesDF = results_df[['latitude', 'longitude']].copy()
treesDF.dropna(subset=['latitude','longitude'], inplace=True)

treesDF

,latitude,longitude
0,40.69199129,-73.82114902
1,40.69161541,-73.82096547
2,40.69153616,-73.82092678
3,40.69210017,-73.82832551
4,40.69166655,-73.829862
5,40.6921299,-73.82821961
6,40.69382431,-73.82753579
7,40.69385554,-73.82742485
8,40.69394083,-73.8271218
9,40.69379328,-73.82764605


In [7]:
df_threeOneOneReq = df_threeOneOneReq[['latitude', 'longitude']].copy()
df_threeOneOneReq.dropna(subset=['latitude','longitude'], inplace=True)

df_threeOneOneReq

,latitude,longitude
0,40.68465116557191,-73.83008135606646
1,40.68405687258566,-73.83285917965487
2,40.690636196427924,-73.8291862515534
3,40.69161321630308,-73.83666620882741
4,40.685837578189094,-73.8268476319625
5,40.685611053802326,-73.83329878273523
6,40.69563833902324,-73.81570779755315
7,40.69359308736717,-73.82970516862927
8,40.68750828614852,-73.81229395493435
9,40.69066509760756,-73.83203848700488


In [9]:
import numpy as np
from pysal.cg.kdtree import KDTree
from pysal.cg import RADIUS_EARTH_MILES

complaints_xys = df_threeOneOneReq[['latitude', 'longitude']].astype(np.float).values 
complaints_tree = KDTree(complaints_xys, distance_metric='Arc', radius=RADIUS_EARTH_MILES)

In [10]:
complaints_count = len(complaints_xys)
complaints_count

617

In [11]:
xy = 40.682460735128025,-73.8300148272251
distances, indices = complaints_tree.query(xy, k=complaints_count, distance_upper_bound=0.5)

In [13]:
indices

indices[~np.isnan(indices)]

len(indices[~np.isnan(indices)])


314

In [14]:
radius_in_miles = 0.0497097

def get_complaint_count(r):
    xy = r['latitude'], r['longitude']
    distances, indices = complaints_tree.query(xy, k=complaints_count, distance_upper_bound=radius_in_miles)
    indices = indices[~np.isnan(indices)]
    return len(indices)

treesDF = treesDF.apply(pd.to_numeric)
treesDF['# of Complaints within 0.5 miles'] = treesDF.apply(get_complaint_count,axis=1)

In [15]:
results_df['complaints'] = treesDF['# of Complaints within 0.5 miles']

In [33]:
results_df
target_folder = '/tmp'
from os.path import join
target_path = join(target_folder, 'results_df.csv')
results_df.to_csv(target_path, index=False)
print('results_df = %s' % target_path)

results_df = /tmp/results_df.csv
